이 튜토리얼은 아래의 튜토리얼을 바탕으로 만들었습니다. 해석의 편의성을 위하여 주석을 추가하였고, hyper parameters 를 바꿔가며 input, hidden, output sequece 의 모습이 어떻게 달라지는지, 그리고 batch processing 을 할 때는 또 어떻게 달라지는지 확인할 수 있는 함수를 추가하였습니다.

https://nbviewer.jupyter.org/github/simonjisu/pytorch_tutorials/blob/master/00_Basic_Utils/02_PackedSequence.ipynb

In [3]:
import torch
import torch.nn as nn

RNN 계열 모델들은 모두 torch.nn.RNNBase 를 상속합니다. `forward_impl` 함수를 살펴보면 hidden vector, hx 가 입력되지 않았을 경우에 이를 zero vector 로 초기화 하는 부분이 있습니다. RNN 과 GRU 는 (sum of all hidden size, num batch, hidden size) 크기의 zero tensor 를 return 하며, hidden 과 memory cell 을 각각 관리하는 LSTM 은 이를 두 개로 복제하여 return 합니다.

```python
class GRU(RNNBase):

    def forward_impl(self, input, hx, batch_sizes, max_batch_size, sorted_indices):
        if hx is None:
            num_directions = 2 if self.bidirectional else 1
            zeros = torch.zeros(self.num_layers * num_directions,
                                max_batch_size, self.hidden_size,
                                dtype=input.dtype, device=input.device)
            hx = zeros
        else:
            hx = self.permute_hidden(hx, sorted_indices)
```

```python
class LSTM(RNNBase):

    def forward_impl(self, input, hx, batch_sizes, max_batch_size, sorted_indices):
        if hx is None:
            num_directions = 2 if self.bidirectional else 1
            zeros = torch.zeros(self.num_layers * num_directions,
                                max_batch_size, self.hidden_size,
                                dtype=input.dtype, device=input.device)
            hx = (zeros, zeros)
        else:
            hx = self.permute_hidden(hx, sorted_indices)
```

입력값의 크기와 hidden size, layer 의 개수, 그리고 batch first 에 따라서 input sequence 를 어떻게 만들어야 하며, 모델에 따라 output sequence 와 hidden vector 의 크기가 어떻게 변하는지 확인하기 위하여 다음의 함수를 만듭니다.

In [1]:
from pprint import pprint

def check_rnn_variable_size(input_size, hidden_size, num_layers,
    batch_size, max_length, mode, batch_first, bidirectional):

    print('{}\nArguments'.format('-'*40))
    for arg, value in locals().items():
        print(' - {} = {}'.format(arg, value))

    # create model
    if mode == 'gru':
        model = nn.GRU(input_size, hidden_size, num_layers,
            bidirectional=bidirectional, batch_first=batch_first)
    elif mode == 'lstm':
        model = nn.LSTM(input_size, hidden_size, num_layers,
            bidirectional=bidirectional, batch_first=batch_first)
    else:
        model = nn.RNN(input_size, hidden_size, num_layers,
            bidirectional=bidirectional, batch_first=batch_first)

    # generate data
    if batch_first:
        inp = torch.randn(batch_size, max_length, input_size)
    else:
        inp = torch.randn(max_length, batch_size, input_size)

    # get out and hidden
    out, hn = model(inp)

    print('\nVariables')
    print(' - in size = {}'.format(inp.size()))
    print(' - out size = {}'.format(out.size()))
    if mode == 'lstm':
        print(' - hn size = ({}, {})'.format(hn[0].size(), hn[1].size()))
    else:
        print(' - hn size = {}'.format(hn.size()))

In [64]:
check_rnn_variable_size(input_size=8, hidden_size=50, num_layers=2,
    batch_size=1, max_length=30, mode='gru', batch_first=True, bidirectional=False)

check_rnn_variable_size(input_size=8, hidden_size=50, num_layers=3,
    batch_size=1, max_length=30, mode='gru', batch_first=True, bidirectional=False)

check_rnn_variable_size(input_size=8, hidden_size=50, num_layers=3,
    batch_size=1, max_length=30, mode='gru', batch_first=True, bidirectional=True)

check_rnn_variable_size(input_size=8, hidden_size=50, num_layers=1,
    batch_size=1, max_length=30, mode='lstm', batch_first=True, bidirectional=False)

check_rnn_variable_size(input_size=8, hidden_size=50, num_layers=3,
    batch_size=1, max_length=30, mode='gru', batch_first=False, bidirectional=True)

----------------------------------------
Arguments
 - input_size = 8
 - hidden_size = 50
 - num_layers = 2
 - batch_size = 1
 - max_length = 30
 - mode = gru
 - batch_first = True
 - bidirectional = False

Variables
 - in size = torch.Size([1, 30, 8])
 - out size = torch.Size([1, 30, 50])
 - hn size = torch.Size([2, 1, 50])
----------------------------------------
Arguments
 - input_size = 8
 - hidden_size = 50
 - num_layers = 3
 - batch_size = 1
 - max_length = 30
 - mode = gru
 - batch_first = True
 - bidirectional = False

Variables
 - in size = torch.Size([1, 30, 8])
 - out size = torch.Size([1, 30, 50])
 - hn size = torch.Size([3, 1, 50])
----------------------------------------
Arguments
 - input_size = 8
 - hidden_size = 50
 - num_layers = 3
 - batch_size = 1
 - max_length = 30
 - mode = gru
 - batch_first = True
 - bidirectional = True

Variables
 - in size = torch.Size([1, 30, 8])
 - out size = torch.Size([1, 30, 100])
 - hn size = torch.Size([6, 1, 50])
----------------------

Batch 단위로 모델을 학습하거나 이용하기 위해서는 input sequence 의 길이를 모두 맞춰줘야 합니다. 그러면 batch data 에 불필요한 부분이 포함됩니다. 해당 부분들은 계산을 굳이 할 필요가 없는 부분임을 알려주기 위하여 input sequence 의 길이를 함께 입력하는데, input sequence 와 sequence lengths 의 길이를 함께 묶은 데이터 형식을 PackedSequence 라 합니다. 이를 만드는 과정과 이용하는 방법에 대하여 알아봅니다.

길이가 각각 5, 2, 3 인 세 개의 input sequence 를 만들었습니다. 이들은 각각 4 차원의 벡터로 구성되어 있습니다. 그리고 lengths 에는 각 sequence 의 길이가 저장되어 있습니다.

In [29]:
input_size = 4
lengths = [5, 2, 3]
inp = [torch.randn(length, input_size)
       for length in lengths]

inp

[tensor([[ 0.3913, -0.4050,  0.3912, -0.2762],
         [ 0.8154,  0.5899, -1.0077, -0.6126],
         [ 0.4029,  0.6696,  0.6185, -0.4720],
         [-0.4939,  0.4012, -0.3134,  0.5090],
         [-0.8124,  0.5563,  1.0831,  0.3560]]),
 tensor([[-0.3416, -0.3998, -1.3086, -0.2742],
         [-1.2784,  0.0398,  0.0912, -0.6887]]),
 tensor([[ 0.4991,  1.0549, -0.3899,  0.4097],
         [ 3.5604,  0.0512,  1.1822,  1.5211],
         [ 0.4519,  1.2172, -0.4908,  0.9669]])]

우리는 입력 데이터의 형식을 (batch, sequence length, input dim) 의 형식으로 만들겠습니다. RNN 은 (sequence length, batch, input dim) 의 형식으로도 만들 수 있습니다. 각 sequence 의 뒷 부분에 zero vector 를 추가하여 모든 sequence 의 길이를 동일하게 만들었습니다. 그리고 이들을 padded_inp 라는 tensor 로 묶었습니다.

In [30]:
max_length = max(lengths)
num_batch = len(lengths)
padded_inp = torch.zeros(num_batch, max_length, input_dim)
for i, (inp_i, length) in enumerate(zip(inp, lengths)):
    padded_inp[i,:length] = inp_i

padded_inp

tensor([[[ 0.3913, -0.4050,  0.3912, -0.2762],
         [ 0.8154,  0.5899, -1.0077, -0.6126],
         [ 0.4029,  0.6696,  0.6185, -0.4720],
         [-0.4939,  0.4012, -0.3134,  0.5090],
         [-0.8124,  0.5563,  1.0831,  0.3560]],

        [[-0.3416, -0.3998, -1.3086, -0.2742],
         [-1.2784,  0.0398,  0.0912, -0.6887],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.4991,  1.0549, -0.3899,  0.4097],
         [ 3.5604,  0.0512,  1.1822,  1.5211],
         [ 0.4519,  1.2172, -0.4908,  0.9669],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]])

pack_padded_sequence 를 이용하면 PackedSequence 를 만들 수 있습니다. 그런데 아래처럼 함수를 실행하면 오류가 발생합니다. lengths 의 기준에 맞춰 batch data 를 내림차순으로 정렬해야 한다고 말합니다.

In [31]:
from torch.nn.utils.rnn import pack_padded_sequence

packed_inp = pack_padded_sequence(padded_inp, lengths, batch_first=True)

RuntimeError: `lengths` array must be sorted in decreasing order when `enforce_sorted` is True. You can pass `enforce_sorted=False` to pack_padded_sequence and/or pack_sequence to sidestep this requirement if you do not need ONNX exportability.

하지만 반드시 직접 정렬할 필요는 없습니다. `enforce_sorted=False` 로 설정하면 함수 내에서 정렬을 해줍니다. 아래의 세번째 출력물, `packed_inp[2]` 를 살펴보면 길이 기준으로 정렬된 sequence 순서는 [0, 2, 1] 입니다. 그리고 PackedSequence 의 첫번째 변수에는 [0, 2, 1] 순서로 위에서 sequence 의 row 를 돌면서 수직 방향으로 concatenate 되어 있음을 확인할 수 있습니다.

또한 batch size 를 살펴보면 [3, 3, 2, 1, 1] 로, PackedSequence 의 data 를 몇 개 단위로 한 번에 forward 시켜야 하는지 확인할 수 있습니다.

In [32]:
packed_inp = pack_padded_sequence(padded_inp, lengths, batch_first=True, enforce_sorted=False)

print(packed_inp[0].size())
print(packed_inp[1].size()) # batch size
print(packed_inp[2]) # sorted_indices
print(packed_inp[3], end='\n\n') # unsorted_indices

print(packed_inp[0], end='\n\n')
print(packed_inp[1])

torch.Size([10, 4])
torch.Size([5])
tensor([0, 2, 1])
tensor([0, 2, 1])

tensor([[ 0.3913, -0.4050,  0.3912, -0.2762],
        [ 0.4991,  1.0549, -0.3899,  0.4097],
        [-0.3416, -0.3998, -1.3086, -0.2742],
        [ 0.8154,  0.5899, -1.0077, -0.6126],
        [ 3.5604,  0.0512,  1.1822,  1.5211],
        [-1.2784,  0.0398,  0.0912, -0.6887],
        [ 0.4029,  0.6696,  0.6185, -0.4720],
        [ 0.4519,  1.2172, -0.4908,  0.9669],
        [-0.4939,  0.4012, -0.3134,  0.5090],
        [-0.8124,  0.5563,  1.0831,  0.3560]])

tensor([3, 3, 2, 1, 1])


조그마한 GRU 모델을 만든 뒤, PackedSequence 를 입력하였을 경우 forward 함수의 return 의 형식을 알아봅니다. 앞서 살펴본 것처럼 zero vector 형식의 initial hidden 을 반드시 입력할 필요는 없습니다. 입력되지 않으면 모델이 스스로 만듭니다.

In [57]:
small_gru = nn.GRU(input_size, hidden_size=5, num_layers=1,
    bidirectional=False, batch_first=True)

packed_out, hn = small_gru(packed_inp)


출력된 PackedSequence 에는 out sequence 가 있습니다. 이는 각 input sequence 의 x_t 에 해당하는 h_t 입니다. x_t 에 대해 분류하는 모델을 학습할 때에는 이를 이용하여 output value 에 대한 loss 를 정의하면 됩니다. out 의 크기는 (sum sequence length, hidden dim) 입니다.

In [62]:
out, out_lengths, _, _ = packed_out

print(out.size(), end='\n\n')
print(out)

torch.Size([10, 5])

tensor([[ 0.2795,  0.0582,  0.0398,  0.0781, -0.2047],
        [-0.1124,  0.1599,  0.2090, -0.0322, -0.3735],
        [ 0.1658,  0.2551,  0.2311, -0.3641, -0.0822],
        [ 0.3981,  0.3509,  0.1124,  0.0375, -0.4616],
        [ 0.2498,  0.0291, -0.2246,  0.0752, -0.7457],
        [ 0.0470,  0.1703,  0.4467, -0.0514, -0.1060],
        [ 0.4298,  0.2726,  0.0858,  0.3408, -0.5715],
        [-0.1363,  0.0435,  0.1118, -0.1293, -0.6678],
        [ 0.0341,  0.0750,  0.2964, -0.0087, -0.4889],
        [-0.2096, -0.1641,  0.3784,  0.1888, -0.4749]], grad_fn=<CatBackward>)


hn 은 각 sequence 의 마지막 지점 t=n 일때의 hidden 인 h_n 입니다. 마지막 hidden vector 를 이용하는 경우에는 이 값을 이용하면 됩니다. 이는 (num_directions * num_layers, batch size, hidden dim) 형식 입니다.

In [61]:
hn.size()

torch.Size([1, 3, 5])

In [60]:
hn

tensor([[[-0.2096, -0.1641,  0.3784,  0.1888, -0.4749],
         [ 0.0470,  0.1703,  0.4467, -0.0514, -0.1060],
         [-0.1363,  0.0435,  0.1118, -0.1293, -0.6678]]],
       grad_fn=<IndexSelectBackward>)

RNN 계열 모델의 return 은 (output, hidden vector) 입니다. 하지만 PackedSequence 를 입력한 경우에는 PackedSequence 형식으로 output 이 출력됩니다. 이를 zero padding 을 한 형식으로 만들기 위해서 `pad_packed_sequence` 를 이용합니다. 0 이 아닌 다른 값으로 값을 넣으려면 `padding_value` 를 설정하면 됩니다.

packed_out 에는 입력된 순서대로 재정렬이 된 output sequence 와 sequence length 가 포함되어 있습니다.

In [46]:
from torch.nn.utils.rnn import pad_packed_sequence

padded_out, padded_out_lengths = pad_packed_sequence(packed_out, batch_first=True, padding_value=0.)
print(padded_out_lengths, end='\n\n')
print(padded_out)

tensor([5, 2, 3])

tensor([[[ 0.1427, -0.2409,  0.2083,  0.0827,  0.1144],
         [ 0.2884,  0.0650,  0.1743, -0.0921,  0.5453],
         [ 0.3122, -0.0597,  0.2542,  0.0661,  0.4575],
         [ 0.4193, -0.2347,  0.4834,  0.0317,  0.1928],
         [ 0.4310, -0.3728,  0.6249,  0.3936, -0.0764]],

        [[ 0.3577, -0.0097,  0.2759, -0.0790,  0.3571],
         [ 0.5906, -0.0666,  0.5160,  0.1937,  0.1906],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.1398, -0.0426,  0.2247, -0.1346,  0.1542],
         [-0.3591, -0.7148, -0.3157, -0.2961,  0.3067],
         [-0.0696, -0.4781,  0.3159, -0.3956,  0.0227],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<IndexSelectBackward>)


두 개의 pack padded sequence 와 pad packed sequence 를 이용하는 batch processing 의 경우 데이터를 어떤 형식으로 만들어야 하는지, 그리고 각각의 변수들의 모양은 어떠한지 확인하기 위하여 다음의 함수를 이용합니다. Input sequence 의 길이는 5부터 최대 길이까지 임의로 정의하였습니다. 즉, 각 sequence 의 length 이후 부분은 일종의 random padding 이라 생각할 수 있습니다.

In [7]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

def check_rnn_variable_size_batch_processing(input_size, hidden_size, num_layers,
    batch_size, max_length, mode, batch_first, bidirectional):

    print('{}\nArguments'.format('-'*40))
    for arg, value in locals().items():
        print(' - {} = {}'.format(arg, value))

    # create model
    if mode == 'gru':
        model = nn.GRU(input_size, hidden_size, num_layers,
            bidirectional=bidirectional, batch_first=batch_first)
    elif mode == 'lstm':
        model = nn.LSTM(input_size, hidden_size, num_layers,
            bidirectional=bidirectional, batch_first=batch_first)
    else:
        model = nn.RNN(input_size, hidden_size, num_layers,
            bidirectional=bidirectional, batch_first=batch_first)

    # generate data
    if batch_first:
        inp = torch.randn(batch_size, max_length, input_size)
    else:
        inp = torch.randn(max_length, batch_size, input_size)

    # generate sequence length [low, high)
    lengths = torch.randint(low=5, high=max_length+1, size=(batch_size,))
    packed_inp =pack_padded_sequence(inp, lengths, batch_first, enforce_sorted=False)

    # apply model
    packed_out, hn = model(packed_inp)

    # pad packed out
    padded_out = pad_packed_sequence(packed_out, batch_first, padding_value=0)
    out, out_lengths = padded_out

    print('\nVariables in packed input')
    print(' - input size = {}'.format(inp.size()))
    print(' - lengths size = {}'.format(lengths.size()))

    print('\nVariables in packed out')
    print(' - out size = {}'.format(packed_out[0].size()))
    print(' - length size = {}'.format(packed_out[1].size()))
    print(' - sorted_indices size = {}'.format(packed_out[2].size()))
    print(' - unsorted_indices size = {}'.format(packed_out[3].size()))

    print('\nVariables in padded out')
    print(' - out size = {}'.format(out.size()))
    print(' - out lengths size = {}'.format(out_lengths.size()))
    print(' - is same (out_lengths, lengths) = {}'.format(
        ( (torch.sum(padded_out[1] == lengths)).numpy() == 512 ) ))
    if mode == 'lstm':
        print(' - hn size = ({}, {}'.format(hn[0].size(), hn[1].size()))
    else:
        print(' - hn size = {}'.format(hn.size()))

In [9]:
check_rnn_variable_size_batch_processing(input_size=8, hidden_size=50, num_layers=3,
    batch_size=512, max_length=30, mode='gru', batch_first=True, bidirectional=True)

check_rnn_variable_size_batch_processing(input_size=8, hidden_size=50, num_layers=3,
    batch_size=512, max_length=30, mode='gru', batch_first=False, bidirectional=True)

check_rnn_variable_size_batch_processing(input_size=8, hidden_size=50, num_layers=3,
    batch_size=512, max_length=30, mode='lstm', batch_first=False, bidirectional=True)

----------------------------------------
Arguments
 - input_size = 8
 - hidden_size = 50
 - num_layers = 3
 - batch_size = 512
 - max_length = 30
 - mode = gru
 - batch_first = True
 - bidirectional = True

Variables in packed input
 - input size = torch.Size([512, 30, 8])
 - lengths size = torch.Size([512])

Variables in packed out
 - out size = torch.Size([8942, 100])
 - length size = torch.Size([30])
 - sorted_indices size = torch.Size([512])
 - unsorted_indices size = torch.Size([512])

Variables in padded out
 - out size = torch.Size([512, 30, 100])
 - out lengths size = torch.Size([512])
 - is same (out_lengths, lengths) = True
 - hn size = torch.Size([6, 512, 50])
----------------------------------------
Arguments
 - input_size = 8
 - hidden_size = 50
 - num_layers = 3
 - batch_size = 512
 - max_length = 30
 - mode = gru
 - batch_first = False
 - bidirectional = True

Variables in packed input
 - input size = torch.Size([30, 512, 8])
 - lengths size = torch.Size([512])

Variable

In [63]:
linear = nn.Linear(in_features=3, out_features=5)
# (batch, 1, in_features)
inp = torch.randn(128, 2, 3)
print(linear(inp).size())

inp = torch.randn(128, 3)
print(linear(inp).size())

torch.Size([128, 2, 5])
torch.Size([128, 5])
